# Análisis de Clustering No Supervisado: Segmentación de Feedback Financiero

## Objetivo del Ejercicio

Aplicar técnicas de **Machine Learning No Supervisado** para identificar patrones y segmentos naturales en el feedback de clientes de productos financieros. Este análisis permite:

- Identificar grupos de clientes con comportamientos y experiencias similares
- Descubrir insights ocultos en datos no etiquetados
- Priorizar acciones de mejora basadas en segmentos críticos
- Generar hipótesis para análisis supervisados posteriores

## Estructura del Notebook

1. **Configuración del Entorno**
2. **Exploración y Preparación de Datos**
3. **Ingeniería de Características**
4. **Análisis de Clustering K-Means**
5. **Clustering Jerárquico**
6. **DBSCAN - Density-Based Clustering**
7. **Comparación de Modelos**
8. **Interpretación de Negocio**
9. **Recomendaciones Estratégicas**

---

## BLOQUE 1: Configuración del Entorno y Librerías

Instalación y carga de todas las dependencias necesarias para el análisis.

In [ ]:
# Importaciones principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configuración de visualización
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

print("✓ Librerías básicas cargadas correctamente")

In [ ]:
# Librerías de Machine Learning
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist, squareform

# Librerías de procesamiento de texto
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import Counter

print("✓ Librerías de ML y NLP cargadas correctamente")

---
## BLOQUE 2: Carga y Exploración Inicial de Datos

Análisis exploratorio para comprender la estructura y calidad del dataset.

In [ ]:
# Carga del dataset
df = pd.read_csv('financial_feedback_data.csv')

print("=" * 80)
print("RESUMEN GENERAL DEL DATASET")
print("=" * 80)
print(f"\nDimensiones: {df.shape[0]} filas x {df.shape[1]} columnas")
print(f"\nPrimeras 5 observaciones:")
display(df.head())

print("\n" + "=" * 80)
print("INFORMACIÓN DE TIPOS DE DATOS")
print("=" * 80)
print(df.info())

In [ ]:
# Análisis de valores faltantes
print("=" * 80)
print("ANÁLISIS DE VALORES FALTANTES")
print("=" * 80)

missing_data = pd.DataFrame({
    'Columna': df.columns,
    'Valores_Faltantes': df.isnull().sum(),
    'Porcentaje': (df.isnull().sum() / len(df) * 100).round(2)
})
missing_data = missing_data[missing_data['Valores_Faltantes'] > 0].sort_values('Valores_Faltantes', ascending=False)

if len(missing_data) > 0:
    display(missing_data)
else:
    print("✓ No se detectaron valores faltantes en el dataset")

# Estadísticas descriptivas de variables numéricas
print("\n" + "=" * 80)
print("ESTADÍSTICAS DESCRIPTIVAS - VARIABLES NUMÉRICAS")
print("=" * 80)
display(df.describe())

In [ ]:
# Análisis de variables categóricas
print("=" * 80)
print("DISTRIBUCIÓN DE VARIABLES CATEGÓRICAS")
print("=" * 80)

categorical_cols = ['Producto', 'Canal_Feedback', 'Cliente_Tipo', 'Región']

for col in categorical_cols:
    print(f"\n{col}:")
    print("-" * 40)
    value_counts = df[col].value_counts()
    percentage = (value_counts / len(df) * 100).round(2)
    result = pd.DataFrame({
        'Frecuencia': value_counts,
        'Porcentaje': percentage
    })
    display(result)

---
## BLOQUE 3: Análisis Exploratorio Visual

Visualizaciones para identificar patrones preliminares en los datos.

In [ ]:
# Distribución de Rating
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Análisis de Distribución de Variables Clave', fontsize=16, fontweight='bold')

# Rating distribution
sns.countplot(data=df, x='Rating', ax=axes[0, 0], palette='viridis')
axes[0, 0].set_title('Distribución de Rating', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Rating')
axes[0, 0].set_ylabel('Frecuencia')

# Añadir valores en las barras
for container in axes[0, 0].containers:
    axes[0, 0].bar_label(container)

# Producto distribution
product_counts = df['Producto'].value_counts()
axes[0, 1].barh(product_counts.index, product_counts.values, color='coral')
axes[0, 1].set_title('Distribución por Producto', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Frecuencia')
axes[0, 1].set_ylabel('Producto')

# Canal Feedback distribution
canal_counts = df['Canal_Feedback'].value_counts()
axes[1, 0].barh(canal_counts.index, canal_counts.values, color='lightblue')
axes[1, 0].set_title('Distribución por Canal de Feedback', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Frecuencia')
axes[1, 0].set_ylabel('Canal')

# Región distribution
sns.countplot(data=df, x='Región', ax=axes[1, 1], palette='Set2')
axes[1, 1].set_title('Distribución por Región', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Región')
axes[1, 1].set_ylabel('Frecuencia')
axes[1, 1].tick_params(axis='x', rotation=45)

for container in axes[1, 1].containers:
    axes[1, 1].bar_label(container)

plt.tight_layout()
plt.show()

In [ ]:
# Análisis de Rating por Producto y Canal
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
fig.suptitle('Rating Promedio por Categorías', fontsize=16, fontweight='bold')

# Rating por Producto
rating_por_producto = df.groupby('Producto')['Rating'].mean().sort_values(ascending=True)
axes[0].barh(rating_por_producto.index, rating_por_producto.values, color='steelblue')
axes[0].set_title('Rating Promedio por Producto', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Rating Promedio')
axes[0].set_xlim(0, 5)
axes[0].axvline(x=df['Rating'].mean(), color='red', linestyle='--', label=f'Media Global: {df["Rating"].mean():.2f}')
axes[0].legend()

# Rating por Canal
rating_por_canal = df.groupby('Canal_Feedback')['Rating'].mean().sort_values(ascending=True)
axes[1].barh(rating_por_canal.index, rating_por_canal.values, color='darkorange')
axes[1].set_title('Rating Promedio por Canal de Feedback', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Rating Promedio')
axes[1].set_xlim(0, 5)
axes[1].axvline(x=df['Rating'].mean(), color='red', linestyle='--', label=f'Media Global: {df["Rating"].mean():.2f}')
axes[1].legend()

plt.tight_layout()
plt.show()

---
## BLOQUE 4: Ingeniería de Características

Transformación de variables categóricas y creación de features numéricas para clustering.

In [ ]:
# Crear copia del dataframe para no modificar el original
df_clustering = df.copy()

# Convertir fecha a datetime y extraer características temporales
df_clustering['Fecha'] = pd.to_datetime(df_clustering['Fecha'])
df_clustering['Mes'] = df_clustering['Fecha'].dt.month
df_clustering['Trimestre'] = df_clustering['Fecha'].dt.quarter
df_clustering['Dia_Semana'] = df_clustering['Fecha'].dt.dayofweek

# Análisis de longitud de comentarios
df_clustering['Longitud_Comentario'] = df_clustering['Comentario'].str.len()
df_clustering['Num_Palabras'] = df_clustering['Comentario'].str.split().str.len()

# Clasificación manual de sentimiento basado en Rating
def clasificar_sentimiento(rating):
    if rating <= 2:
        return 'Negativo'
    elif rating == 3:
        return 'Neutral'
    else:
        return 'Positivo'

df_clustering['Sentimiento_Derivado'] = df_clustering['Rating'].apply(clasificar_sentimiento)

print("✓ Características temporales y de texto creadas")
print(f"\nNuevas características:")
print(f"  - Mes, Trimestre, Dia_Semana")
print(f"  - Longitud_Comentario, Num_Palabras")
print(f"  - Sentimiento_Derivado")

display(df_clustering[['ID_Comentario', 'Rating', 'Mes', 'Trimestre', 
                        'Longitud_Comentario', 'Num_Palabras', 'Sentimiento_Derivado']].head(10))

In [ ]:
# Encoding de variables categóricas
print("=" * 80)
print("ENCODING DE VARIABLES CATEGÓRICAS")
print("=" * 80)

# Label Encoding para variables con cardinalidad baja
le_dict = {}
categorical_features = ['Producto', 'Canal_Feedback', 'Cliente_Tipo', 'Región', 'Sentimiento_Derivado']

for feature in categorical_features:
    le = LabelEncoder()
    df_clustering[f'{feature}_Encoded'] = le.fit_transform(df_clustering[feature])
    le_dict[feature] = le
    print(f"\n{feature}:")
    print(f"  Clases: {list(le.classes_)}")
    print(f"  Encodings: {list(range(len(le.classes_)))}")

print("\n✓ Variables categóricas codificadas exitosamente")

In [ ]:
# Selección de features para clustering
features_for_clustering = [
    'Rating',
    'Producto_Encoded',
    'Canal_Feedback_Encoded',
    'Cliente_Tipo_Encoded',
    'Región_Encoded',
    'Mes',
    'Trimestre',
    'Longitud_Comentario',
    'Num_Palabras',
    'Sentimiento_Derivado_Encoded'
]

X = df_clustering[features_for_clustering].copy()

print("=" * 80)
print("MATRIZ DE CARACTERÍSTICAS PARA CLUSTERING")
print("=" * 80)
print(f"\nDimensiones: {X.shape}")
print(f"\nFeatures seleccionadas:")
for i, feat in enumerate(features_for_clustering, 1):
    print(f"  {i}. {feat}")

print("\nEstadísticas de la matriz de features:")
display(X.describe())

In [ ]:
# Estandarización de features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=features_for_clustering)

print("=" * 80)
print("ESTANDARIZACIÓN DE CARACTERÍSTICAS")
print("=" * 80)
print("\n✓ Features estandarizadas con StandardScaler (media=0, std=1)")
print(f"\nDimensiones post-escalamiento: {X_scaled.shape}")
print("\nPrimeras 5 observaciones estandarizadas:")
display(X_scaled_df.head())

print("\nValidación de estandarización:")
print(f"  Media de features: {X_scaled.mean(axis=0).round(4)}")
print(f"  Desv. Est. de features: {X_scaled.std(axis=0).round(4)}")

---
## BLOQUE 5: Reducción Dimensional con PCA

Aplicación de PCA para visualización y análisis de varianza explicada.

In [ ]:
# PCA para visualización
pca = PCA(n_components=min(10, X_scaled.shape[1]))
X_pca = pca.fit_transform(X_scaled)

print("=" * 80)
print("ANÁLISIS DE COMPONENTES PRINCIPALES (PCA)")
print("=" * 80)

# Varianza explicada
variance_df = pd.DataFrame({
    'Componente': [f'PC{i+1}' for i in range(len(pca.explained_variance_ratio_))],
    'Varianza_Explicada': pca.explained_variance_ratio_,
    'Varianza_Acumulada': np.cumsum(pca.explained_variance_ratio_)
})

print("\nVarianza explicada por componente:")
display(variance_df)

# Visualización de varianza explicada
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Scree plot
axes[0].bar(range(1, len(pca.explained_variance_ratio_) + 1), 
            pca.explained_variance_ratio_, 
            color='steelblue', alpha=0.7)
axes[0].plot(range(1, len(pca.explained_variance_ratio_) + 1), 
             pca.explained_variance_ratio_, 
             marker='o', color='red', linewidth=2)
axes[0].set_xlabel('Componente Principal', fontsize=12)
axes[0].set_ylabel('Varianza Explicada', fontsize=12)
axes[0].set_title('Scree Plot - Varianza Explicada por Componente', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Varianza acumulada
axes[1].plot(range(1, len(pca.explained_variance_ratio_) + 1), 
             np.cumsum(pca.explained_variance_ratio_), 
             marker='o', linewidth=2, color='darkgreen')
axes[1].axhline(y=0.8, color='red', linestyle='--', label='80% Varianza')
axes[1].axhline(y=0.9, color='orange', linestyle='--', label='90% Varianza')
axes[1].set_xlabel('Número de Componentes', fontsize=12)
axes[1].set_ylabel('Varianza Acumulada', fontsize=12)
axes[1].set_title('Varianza Acumulada por Componentes', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✓ Para capturar el 80% de la varianza se requieren {np.argmax(np.cumsum(pca.explained_variance_ratio_) >= 0.8) + 1} componentes")
print(f"✓ Para capturar el 90% de la varianza se requieren {np.argmax(np.cumsum(pca.explained_variance_ratio_) >= 0.9) + 1} componentes")

---
## BLOQUE 6: Método del Codo y Métricas de Validación

Determinación del número óptimo de clusters usando múltiples criterios.

In [ ]:
# Método del Codo (Elbow Method) y métricas de validación
print("=" * 80)
print("EVALUACIÓN DEL NÚMERO ÓPTIMO DE CLUSTERS")
print("=" * 80)
print("\nCalculando métricas para k=2 hasta k=10...\n")

inertias = []
silhouette_scores = []
davies_bouldin_scores = []
calinski_harabasz_scores = []

K_range = range(2, 11)

for k in K_range:
    kmeans_temp = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels_temp = kmeans_temp.fit_predict(X_scaled)
    
    inertias.append(kmeans_temp.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, labels_temp))
    davies_bouldin_scores.append(davies_bouldin_score(X_scaled, labels_temp))
    calinski_harabasz_scores.append(calinski_harabasz_score(X_scaled, labels_temp))
    
    print(f"k={k}: Inercia={kmeans_temp.inertia_:.2f}, "
          f"Silhouette={silhouette_scores[-1]:.3f}, "
          f"Davies-Bouldin={davies_bouldin_scores[-1]:.3f}")

print("\n✓ Cálculo completado")

In [ ]:
# Visualización de métricas de evaluación
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Métricas de Evaluación para Selección de K Óptimo', fontsize=16, fontweight='bold')

# Método del Codo (Inertia)
axes[0, 0].plot(K_range, inertias, marker='o', linewidth=2, markersize=8, color='steelblue')
axes[0, 0].set_xlabel('Número de Clusters (k)', fontsize=11)
axes[0, 0].set_ylabel('Inercia (Within-Cluster Sum of Squares)', fontsize=11)
axes[0, 0].set_title('Método del Codo - Inercia', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].set_xticks(K_range)

# Silhouette Score (más alto es mejor)
axes[0, 1].plot(K_range, silhouette_scores, marker='s', linewidth=2, markersize=8, color='green')
axes[0, 1].set_xlabel('Número de Clusters (k)', fontsize=11)
axes[0, 1].set_ylabel('Silhouette Score', fontsize=11)
axes[0, 1].set_title('Silhouette Score (↑ mejor)', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].set_xticks(K_range)
best_k_silhouette = K_range[np.argmax(silhouette_scores)]
axes[0, 1].axvline(x=best_k_silhouette, color='red', linestyle='--', alpha=0.7, label=f'Mejor k={best_k_silhouette}')
axes[0, 1].legend()

# Davies-Bouldin Index (más bajo es mejor)
axes[1, 0].plot(K_range, davies_bouldin_scores, marker='^', linewidth=2, markersize=8, color='orange')
axes[1, 0].set_xlabel('Número de Clusters (k)', fontsize=11)
axes[1, 0].set_ylabel('Davies-Bouldin Index', fontsize=11)
axes[1, 0].set_title('Davies-Bouldin Index (↓ mejor)', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_xticks(K_range)
best_k_db = K_range[np.argmin(davies_bouldin_scores)]
axes[1, 0].axvline(x=best_k_db, color='red', linestyle='--', alpha=0.7, label=f'Mejor k={best_k_db}')
axes[1, 0].legend()

# Calinski-Harabasz Index (más alto es mejor)
axes[1, 1].plot(K_range, calinski_harabasz_scores, marker='D', linewidth=2, markersize=8, color='purple')
axes[1, 1].set_xlabel('Número de Clusters (k)', fontsize=11)
axes[1, 1].set_ylabel('Calinski-Harabasz Index', fontsize=11)
axes[1, 1].set_title('Calinski-Harabasz Index (↑ mejor)', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_xticks(K_range)
best_k_ch = K_range[np.argmax(calinski_harabasz_scores)]
axes[1, 1].axvline(x=best_k_ch, color='red', linestyle='--', alpha=0.7, label=f'Mejor k={best_k_ch}')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

print("\n" + "=" * 80)
print("RECOMENDACIONES DE K ÓPTIMO")
print("=" * 80)
print(f"\nSilhouette Score recomienda: k = {best_k_silhouette}")
print(f"Davies-Bouldin Index recomienda: k = {best_k_db}")
print(f"Calinski-Harabasz Index recomienda: k = {best_k_ch}")
print(f"\n✓ Considere k={best_k_silhouette} como punto de partida óptimo")

---
## BLOQUE 7: Implementación de K-Means Clustering

Aplicación del algoritmo K-Means con el número óptimo de clusters.

In [ ]:
# Aplicar K-Means con k óptimo
optimal_k = best_k_silhouette  # Puede ajustarse manualmente si se desea

print("=" * 80)
print(f"APLICANDO K-MEANS CON K={optimal_k}")
print("=" * 80)

kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=20, max_iter=300)
df_clustering['Cluster_KMeans'] = kmeans_final.fit_predict(X_scaled)

print(f"\n✓ Modelo K-Means entrenado exitosamente")
print(f"\nDistribución de clusters:")
cluster_distribution = df_clustering['Cluster_KMeans'].value_counts().sort_index()
for cluster_id, count in cluster_distribution.items():
    percentage = (count / len(df_clustering) * 100)
    print(f"  Cluster {cluster_id}: {count} observaciones ({percentage:.1f}%)")

# Métricas finales
final_silhouette = silhouette_score(X_scaled, df_clustering['Cluster_KMeans'])
final_db = davies_bouldin_score(X_scaled, df_clustering['Cluster_KMeans'])
final_ch = calinski_harabasz_score(X_scaled, df_clustering['Cluster_KMeans'])

print(f"\nMétricas del modelo final:")
print(f"  Silhouette Score: {final_silhouette:.4f}")
print(f"  Davies-Bouldin Index: {final_db:.4f}")
print(f"  Calinski-Harabasz Index: {final_ch:.2f}")
print(f"  Inercia: {kmeans_final.inertia_:.2f}")

In [ ]:
# Visualización de clusters en espacio PCA
fig, axes = plt.subplots(1, 2, figsize=(18, 6))
fig.suptitle(f'Visualización de Clusters K-Means (k={optimal_k}) en Espacio PCA', 
             fontsize=16, fontweight='bold')

# PC1 vs PC2
scatter1 = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], 
                           c=df_clustering['Cluster_KMeans'], 
                           cmap='viridis', s=50, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[0].set_xlabel('Componente Principal 1', fontsize=11)
axes[0].set_ylabel('Componente Principal 2', fontsize=11)
axes[0].set_title('Clusters en PC1 vs PC2', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=axes[0], label='Cluster')

# PC2 vs PC3
scatter2 = axes[1].scatter(X_pca[:, 1], X_pca[:, 2], 
                           c=df_clustering['Cluster_KMeans'], 
                           cmap='viridis', s=50, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[1].set_xlabel('Componente Principal 2', fontsize=11)
axes[1].set_ylabel('Componente Principal 3', fontsize=11)
axes[1].set_title('Clusters en PC2 vs PC3', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=axes[1], label='Cluster')

plt.tight_layout()
plt.show()

---
## BLOQUE 8: Perfilamiento de Clusters

Análisis detallado de las características de cada cluster.

In [ ]:
# Análisis de centroides
print("=" * 80)
print("ANÁLISIS DE CENTROIDES - K-MEANS")
print("=" * 80)

centroids_df = pd.DataFrame(
    scaler.inverse_transform(kmeans_final.cluster_centers_),
    columns=features_for_clustering
)
centroids_df.index = [f'Cluster {i}' for i in range(optimal_k)]

print("\nCentroides de cada cluster (valores originales):")
display(centroids_df.round(2))

In [ ]:
# Perfilamiento detallado por cluster
print("\n" + "=" * 80)
print("PERFILAMIENTO DETALLADO POR CLUSTER")
print("=" * 80)

for cluster_id in sorted(df_clustering['Cluster_KMeans'].unique()):
    cluster_data = df_clustering[df_clustering['Cluster_KMeans'] == cluster_id]
    
    print(f"\n{'=' * 80}")
    print(f"CLUSTER {cluster_id} - {len(cluster_data)} observaciones ({len(cluster_data)/len(df_clustering)*100:.1f}%)")
    print(f"{'=' * 80}")
    
    print(f"\nRating Promedio: {cluster_data['Rating'].mean():.2f} (±{cluster_data['Rating'].std():.2f})")
    print(f"Longitud Promedio de Comentario: {cluster_data['Longitud_Comentario'].mean():.1f} caracteres")
    
    print(f"\nDistribución de Sentimiento:")
    sentimiento_dist = cluster_data['Sentimiento_Derivado'].value_counts(normalize=True) * 100
    for sent, pct in sentimiento_dist.items():
        print(f"  {sent}: {pct:.1f}%")
    
    print(f"\nTop 3 Productos:")
    top_productos = cluster_data['Producto'].value_counts().head(3)
    for prod, count in top_productos.items():
        print(f"  {prod}: {count} ({count/len(cluster_data)*100:.1f}%)")
    
    print(f"\nTop 3 Canales:")
    top_canales = cluster_data['Canal_Feedback'].value_counts().head(3)
    for canal, count in top_canales.items():
        print(f"  {canal}: {count} ({count/len(cluster_data)*100:.1f}%)")
    
    print(f"\nDistribución por Tipo de Cliente:")
    tipo_dist = cluster_data['Cliente_Tipo'].value_counts()
    for tipo, count in tipo_dist.items():
        print(f"  {tipo}: {count} ({count/len(cluster_data)*100:.1f}%)")
    
    print(f"\nRegiones principales:")
    region_dist = cluster_data['Región'].value_counts().head(3)
    for region, count in region_dist.items():
        print(f"  {region}: {count} ({count/len(cluster_data)*100:.1f}%)")

In [ ]:
# Heatmap de características promedio por cluster
cluster_profiles = df_clustering.groupby('Cluster_KMeans')[features_for_clustering].mean()

plt.figure(figsize=(14, 6))
sns.heatmap(cluster_profiles.T, annot=True, fmt='.2f', cmap='RdYlGn', 
            linewidths=0.5, cbar_kws={'label': 'Valor Promedio'})
plt.title('Perfil de Características por Cluster (K-Means)', fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Cluster', fontsize=12)
plt.ylabel('Característica', fontsize=12)
plt.tight_layout()
plt.show()

---
## BLOQUE 9: Clustering Jerárquico

Aplicación de clustering jerárquico aglomerativo para comparación.

In [ ]:
# Clustering Jerárquico - Dendrograma
print("=" * 80)
print("CLUSTERING JERÁRQUICO AGLOMERATIVO")
print("=" * 80)

# Calcular linkage - usamos una muestra para eficiencia
sample_size = min(500, len(X_scaled))
sample_indices = np.random.choice(len(X_scaled), sample_size, replace=False)
X_sample = X_scaled[sample_indices]

print(f"\nCalculando dendrograma con {sample_size} observaciones muestreadas...")

linkage_matrix = linkage(X_sample, method='ward')

plt.figure(figsize=(16, 8))
dendrogram(linkage_matrix, truncate_mode='lastp', p=20, leaf_font_size=10)
plt.title('Dendrograma - Clustering Jerárquico (Ward)', fontsize=14, fontweight='bold')
plt.xlabel('Índice de Muestra / Tamaño del Cluster', fontsize=12)
plt.ylabel('Distancia', fontsize=12)
plt.axhline(y=50, color='red', linestyle='--', label='Corte sugerido')
plt.legend()
plt.tight_layout()
plt.show()

print("\n✓ Dendrograma generado")

In [ ]:
# Aplicar clustering jerárquico al dataset completo
n_clusters_hierarchical = optimal_k

hierarchical = AgglomerativeClustering(n_clusters=n_clusters_hierarchical, linkage='ward')
df_clustering['Cluster_Hierarchical'] = hierarchical.fit_predict(X_scaled)

print("=" * 80)
print(f"CLUSTERING JERÁRQUICO CON k={n_clusters_hierarchical}")
print("=" * 80)

print(f"\n✓ Modelo aplicado exitosamente")
print(f"\nDistribución de clusters jerárquicos:")
hierarchical_distribution = df_clustering['Cluster_Hierarchical'].value_counts().sort_index()
for cluster_id, count in hierarchical_distribution.items():
    percentage = (count / len(df_clustering) * 100)
    print(f"  Cluster {cluster_id}: {count} observaciones ({percentage:.1f}%)")

# Métricas
hierarchical_silhouette = silhouette_score(X_scaled, df_clustering['Cluster_Hierarchical'])
hierarchical_db = davies_bouldin_score(X_scaled, df_clustering['Cluster_Hierarchical'])
hierarchical_ch = calinski_harabasz_score(X_scaled, df_clustering['Cluster_Hierarchical'])

print(f"\nMétricas del modelo jerárquico:")
print(f"  Silhouette Score: {hierarchical_silhouette:.4f}")
print(f"  Davies-Bouldin Index: {hierarchical_db:.4f}")
print(f"  Calinski-Harabasz Index: {hierarchical_ch:.2f}")

In [ ]:
# Visualización de clusters jerárquicos en PCA
fig, axes = plt.subplots(1, 2, figsize=(18, 6))
fig.suptitle(f'Visualización de Clustering Jerárquico (k={n_clusters_hierarchical}) en Espacio PCA', 
             fontsize=16, fontweight='bold')

scatter1 = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], 
                           c=df_clustering['Cluster_Hierarchical'], 
                           cmap='plasma', s=50, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[0].set_xlabel('Componente Principal 1', fontsize=11)
axes[0].set_ylabel('Componente Principal 2', fontsize=11)
axes[0].set_title('Clusters Jerárquicos en PC1 vs PC2', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=axes[0], label='Cluster')

scatter2 = axes[1].scatter(X_pca[:, 1], X_pca[:, 2], 
                           c=df_clustering['Cluster_Hierarchical'], 
                           cmap='plasma', s=50, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[1].set_xlabel('Componente Principal 2', fontsize=11)
axes[1].set_ylabel('Componente Principal 3', fontsize=11)
axes[1].set_title('Clusters Jerárquicos en PC2 vs PC3', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=axes[1], label='Cluster')

plt.tight_layout()
plt.show()

---
## BLOQUE 10: DBSCAN - Density-Based Clustering

Aplicación de DBSCAN para identificar clusters de densidad variable y outliers.

In [ ]:
# DBSCAN
print("=" * 80)
print("DBSCAN - DENSITY-BASED SPATIAL CLUSTERING")
print("=" * 80)

# Probar diferentes valores de epsilon
print("\nBuscando parámetros óptimos de DBSCAN...")

dbscan = DBSCAN(eps=2.0, min_samples=10)
df_clustering['Cluster_DBSCAN'] = dbscan.fit_predict(X_scaled)

n_clusters_dbscan = len(set(df_clustering['Cluster_DBSCAN'])) - (1 if -1 in df_clustering['Cluster_DBSCAN'] else 0)
n_noise = list(df_clustering['Cluster_DBSCAN']).count(-1)

print(f"\n✓ DBSCAN aplicado con eps=2.0, min_samples=10")
print(f"\nResultados:")
print(f"  Número de clusters encontrados: {n_clusters_dbscan}")
print(f"  Puntos clasificados como ruido (outliers): {n_noise} ({n_noise/len(df_clustering)*100:.1f}%)")

if n_clusters_dbscan > 0:
    print(f"\nDistribución de clusters DBSCAN:")
    dbscan_distribution = df_clustering[df_clustering['Cluster_DBSCAN'] != -1]['Cluster_DBSCAN'].value_counts().sort_index()
    for cluster_id, count in dbscan_distribution.items():
        percentage = (count / len(df_clustering) * 100)
        print(f"  Cluster {cluster_id}: {count} observaciones ({percentage:.1f}%)")

    # Métricas (excluyendo ruido)
    df_dbscan_no_noise = df_clustering[df_clustering['Cluster_DBSCAN'] != -1]
    X_dbscan_no_noise = X_scaled[df_clustering['Cluster_DBSCAN'] != -1]
    
    if len(df_dbscan_no_noise) > 0 and n_clusters_dbscan > 1:
        dbscan_silhouette = silhouette_score(X_dbscan_no_noise, df_dbscan_no_noise['Cluster_DBSCAN'])
        dbscan_db = davies_bouldin_score(X_dbscan_no_noise, df_dbscan_no_noise['Cluster_DBSCAN'])
        dbscan_ch = calinski_harabasz_score(X_dbscan_no_noise, df_dbscan_no_noise['Cluster_DBSCAN'])
        
        print(f"\nMétricas del modelo DBSCAN (sin ruido):")
        print(f"  Silhouette Score: {dbscan_silhouette:.4f}")
        print(f"  Davies-Bouldin Index: {dbscan_db:.4f}")
        print(f"  Calinski-Harabasz Index: {dbscan_ch:.2f}")
else:
    print("\n⚠ DBSCAN no encontró clusters significativos con estos parámetros")

In [ ]:
# Visualización DBSCAN
if n_clusters_dbscan > 0:
    fig, axes = plt.subplots(1, 2, figsize=(18, 6))
    fig.suptitle(f'Visualización de DBSCAN ({n_clusters_dbscan} clusters, {n_noise} outliers) en Espacio PCA', 
                 fontsize=16, fontweight='bold')
    
    # Crear colormap personalizado con color especial para outliers
    scatter1 = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], 
                               c=df_clustering['Cluster_DBSCAN'], 
                               cmap='tab10', s=50, alpha=0.6, edgecolors='black', linewidth=0.5)
    axes[0].set_xlabel('Componente Principal 1', fontsize=11)
    axes[0].set_ylabel('Componente Principal 2', fontsize=11)
    axes[0].set_title('DBSCAN en PC1 vs PC2 (ruido en -1)', fontsize=12, fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    plt.colorbar(scatter1, ax=axes[0], label='Cluster')
    
    scatter2 = axes[1].scatter(X_pca[:, 1], X_pca[:, 2], 
                               c=df_clustering['Cluster_DBSCAN'], 
                               cmap='tab10', s=50, alpha=0.6, edgecolors='black', linewidth=0.5)
    axes[1].set_xlabel('Componente Principal 2', fontsize=11)
    axes[1].set_ylabel('Componente Principal 3', fontsize=11)
    axes[1].set_title('DBSCAN en PC2 vs PC3 (ruido en -1)', fontsize=12, fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    plt.colorbar(scatter2, ax=axes[1], label='Cluster')
    
    plt.tight_layout()
    plt.show()

---
## BLOQUE 11: Comparación de Métodos de Clustering

Análisis comparativo de los tres algoritmos implementados.

In [ ]:
# Tabla comparativa de métricas
print("=" * 80)
print("COMPARACIÓN DE MÉTODOS DE CLUSTERING")
print("=" * 80)

comparison_data = {
    'Método': ['K-Means', 'Jerárquico', 'DBSCAN'],
    'N° Clusters': [
        optimal_k,
        n_clusters_hierarchical,
        n_clusters_dbscan
    ],
    'Silhouette Score': [
        final_silhouette,
        hierarchical_silhouette,
        dbscan_silhouette if n_clusters_dbscan > 1 else np.nan
    ],
    'Davies-Bouldin Index': [
        final_db,
        hierarchical_db,
        dbscan_db if n_clusters_dbscan > 1 else np.nan
    ],
    'Calinski-Harabasz Index': [
        final_ch,
        hierarchical_ch,
        dbscan_ch if n_clusters_dbscan > 1 else np.nan
    ],
    'Outliers Detectados': [
        0,
        0,
        n_noise
    ]
}

comparison_df = pd.DataFrame(comparison_data)
display(comparison_df)

print("\nInterpretación de métricas:")
print("  • Silhouette Score: Más alto es mejor (rango: -1 a 1)")
print("  • Davies-Bouldin Index: Más bajo es mejor (rango: 0 a ∞)")
print("  • Calinski-Harabasz Index: Más alto es mejor (rango: 0 a ∞)")

In [ ]:
# Visualización comparativa
fig, axes = plt.subplots(1, 3, figsize=(20, 5))
fig.suptitle('Comparación Visual de Métodos de Clustering en PC1 vs PC2', 
             fontsize=16, fontweight='bold')

# K-Means
scatter1 = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], 
                           c=df_clustering['Cluster_KMeans'], 
                           cmap='viridis', s=40, alpha=0.6, edgecolors='black', linewidth=0.3)
axes[0].set_title(f'K-Means (k={optimal_k})', fontsize=12, fontweight='bold')
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')
axes[0].grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=axes[0])

# Jerárquico
scatter2 = axes[1].scatter(X_pca[:, 0], X_pca[:, 1], 
                           c=df_clustering['Cluster_Hierarchical'], 
                           cmap='plasma', s=40, alpha=0.6, edgecolors='black', linewidth=0.3)
axes[1].set_title(f'Jerárquico (k={n_clusters_hierarchical})', fontsize=12, fontweight='bold')
axes[1].set_xlabel('PC1')
axes[1].set_ylabel('PC2')
axes[1].grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=axes[1])

# DBSCAN
scatter3 = axes[2].scatter(X_pca[:, 0], X_pca[:, 1], 
                           c=df_clustering['Cluster_DBSCAN'], 
                           cmap='tab10', s=40, alpha=0.6, edgecolors='black', linewidth=0.3)
axes[2].set_title(f'DBSCAN ({n_clusters_dbscan} clusters, {n_noise} outliers)', fontsize=12, fontweight='bold')
axes[2].set_xlabel('PC1')
axes[2].set_ylabel('PC2')
axes[2].grid(True, alpha=0.3)
plt.colorbar(scatter3, ax=axes[2])

plt.tight_layout()
plt.show()

---
## BLOQUE 12: Interpretación de Negocio - Insights Accionables

Traducción de resultados técnicos a recomendaciones estratégicas.

In [ ]:
# Análisis de palabras clave por cluster (K-Means)
print("=" * 80)
print("ANÁLISIS DE PALABRAS CLAVE POR CLUSTER (K-MEANS)")
print("=" * 80)

for cluster_id in sorted(df_clustering['Cluster_KMeans'].unique()):
    cluster_comments = df_clustering[df_clustering['Cluster_KMeans'] == cluster_id]['Comentario']
    
    # Unir todos los comentarios
    all_text = ' '.join(cluster_comments.astype(str))
    
    # Tokenización simple
    words = all_text.lower().split()
    
    # Filtrar palabras comunes (stopwords básico)
    stopwords = ['el', 'la', 'de', 'y', 'a', 'en', 'es', 'por', 'con', 'los', 'las',
                 'del', 'un', 'una', 'para', 'al', 'que', 'se', 'su', 'me', 'mi',
                 'muy', 'pero', 'más', 'son', 'como', 'lo', 'le', 'ha']
    words_filtered = [w for w in words if w not in stopwords and len(w) > 3]
    
    # Contar frecuencias
    word_freq = Counter(words_filtered)
    top_words = word_freq.most_common(10)
    
    print(f"\nCluster {cluster_id}:")
    print("-" * 40)
    for word, freq in top_words:
        print(f"  {word}: {freq} menciones")

In [ ]:
# Dashboard de insights por cluster
print("\n" + "=" * 80)
print("RESUMEN EJECUTIVO - INSIGHTS POR CLUSTER")
print("=" * 80)

for cluster_id in sorted(df_clustering['Cluster_KMeans'].unique()):
    cluster_data = df_clustering[df_clustering['Cluster_KMeans'] == cluster_id]
    
    # Calcular métricas clave
    avg_rating = cluster_data['Rating'].mean()
    size = len(cluster_data)
    size_pct = size / len(df_clustering) * 100
    sentiment_mode = cluster_data['Sentimiento_Derivado'].mode()[0]
    top_product = cluster_data['Producto'].mode()[0]
    top_channel = cluster_data['Canal_Feedback'].mode()[0]
    new_clients_pct = (cluster_data['Cliente_Tipo'] == 'Nuevo').sum() / len(cluster_data) * 100
    
    print(f"\n╔{'═' * 78}╗")
    print(f"║ CLUSTER {cluster_id}: {sentiment_mode.upper()} - {size} clientes ({size_pct:.1f}%)" + " " * (78 - len(f" CLUSTER {cluster_id}: {sentiment_mode.upper()} - {size} clientes ({size_pct:.1f}%)") - 1) + "║")
    print(f"╠{'═' * 78}╣")
    
    # Perfil
    print(f"║ 📊 Rating Promedio: {avg_rating:.2f}/5.0" + " " * (78 - len(f" 📊 Rating Promedio: {avg_rating:.2f}/5.0") - 1) + "║")
    print(f"║ 🎯 Producto Principal: {top_product}" + " " * (78 - len(f" 🎯 Producto Principal: {top_product}") - 1) + "║")
    print(f"║ 📱 Canal Predominante: {top_channel}" + " " * (78 - len(f" 📱 Canal Predominante: {top_channel}") - 1) + "║")
    print(f"║ 🆕 Clientes Nuevos: {new_clients_pct:.1f}%" + " " * (78 - len(f" 🆕 Clientes Nuevos: {new_clients_pct:.1f}%") - 1) + "║")
    
    # Caracterización
    if avg_rating >= 4.0:
        caracterizacion = "Clientes SATISFECHOS - Prioridad: RETENCIÓN"
    elif avg_rating >= 3.0:
        caracterizacion = "Clientes NEUTRALES - Prioridad: MEJORA CONTINUA"
    else:
        caracterizacion = "Clientes EN RIESGO - Prioridad: INTERVENCIÓN INMEDIATA"
    
    print(f"║ ⚠️ {caracterizacion}" + " " * (78 - len(f" ⚠️ {caracterizacion}") - 1) + "║")
    print(f"╚{'═' * 78}╝")

---
## BLOQUE 13: Exportación de Resultados

Generación de datasets con asignaciones de clusters para análisis posteriores.

In [ ]:
# Exportar dataset completo con clusters asignados
output_columns = [
    'ID_Comentario', 'Fecha', 'Producto', 'Canal_Feedback', 'Comentario',
    'Rating', 'Cliente_Tipo', 'Región', 
    'Cluster_KMeans', 'Cluster_Hierarchical', 'Cluster_DBSCAN',
    'Sentimiento_Derivado', 'Longitud_Comentario', 'Num_Palabras'
]

df_export = df_clustering[output_columns].copy()

# Guardar a CSV
output_filename = 'feedback_con_clusters.csv'
df_export.to_csv(output_filename, index=False, encoding='utf-8-sig')

print("=" * 80)
print("EXPORTACIÓN DE RESULTADOS")
print("=" * 80)
print(f"\n✓ Dataset exportado exitosamente: {output_filename}")
print(f"  Dimensiones: {df_export.shape[0]} filas x {df_export.shape[1]} columnas")
print(f"\nColumnas exportadas:")
for col in output_columns:
    print(f"  • {col}")

# Resumen estadístico del export
print("\nPrimeras 10 filas del dataset exportado:")
display(df_export.head(10))

---
## BLOQUE 14: Conclusiones y Recomendaciones Estratégicas

Síntesis ejecutiva para stakeholders y próximos pasos.

In [ ]:
print("="*80)
print("CONCLUSIONES Y RECOMENDACIONES ESTRATÉGICAS")
print("="*80)

print("""
╔════════════════════════════════════════════════════════════════════════════════╗
║                           RESUMEN EJECUTIVO                                    ║
╠════════════════════════════════════════════════════════════════════════════════╣
║                                                                                ║
║  1. HALLAZGOS PRINCIPALES                                                     ║
║     • Se identificaron {} clusters significativos de clientes                 ║
║     • K-Means demostró ser el método más balanceado (Silhouette: {:.3f})      ║
║     • Se detectaron {} outliers mediante DBSCAN                               ║
║                                                                                ║
║  2. SEGMENTOS CLAVE IDENTIFICADOS                                             ║
║     • Analizar cada cluster para entender patrones de satisfacción           ║
║     • Priorizar acciones en clusters con bajo rating                          ║
║     • Replicar prácticas exitosas de clusters con alto rating                ║
║                                                                                ║
║  3. RECOMENDACIONES TÁCTICAS                                                  ║
║     ✓ Intervención inmediata en clusters con Rating < 3.0                     ║
║     ✓ Programas de fidelización para clusters de alto valor                  ║
║     ✓ Optimización de canales según preferencias por cluster                 ║
║     ✓ Personalización de productos por segmento                               ║
║                                                                                ║
║  4. PRÓXIMOS PASOS                                                            ║
║     → Análisis de sentimiento textual con NLP avanzado                        ║
║     → Modelos predictivos de churn por cluster                                ║
║     → Cálculo de Customer Lifetime Value (CLV) segmentado                     ║
║     → Dashboards interactivos para monitoreo continuo                         ║
║                                                                                ║
╚════════════════════════════════════════════════════════════════════════════════╝
""".format(
    optimal_k,
    final_silhouette,
    n_noise
))

print("\n" + "="*80)
print("EJERCICIO COMPLETADO EXITOSAMENTE")
print("="*80)
print("\n✓ Todos los bloques ejecutados")
print("✓ Modelos de clustering entrenados y evaluados")
print("✓ Insights de negocio generados")
print("✓ Resultados exportados")
print("\nEl análisis está listo para ser presentado a stakeholders.")